In [ ]:
import pandas as pd

In [ ]:
df_val = pd.read_csv("dev (1).csv")

df_sagor = pd.read_csv("/content/sagor_dev.csv")
df_sagor_sub = pd.read_csv("/content/sagor_dev_sub.csv")

df_mbert = pd.read_csv("/content/mbert_dev.csv")
df_mbert_sub = pd.read_csv("/content/mbert_dev_sub.csv")

df_roberta = pd.read_csv("/content/roberta_dev.csv")
df_roberta_sub = pd.read_csv("/content/roberta_dev_sub.csv")

df_mlm = pd.read_csv("/content/mlm_dev.csv")

In [ ]:
df_val['sagor'] = df_sagor['label']
# df_val['sagor_sub'] = df_sagor_sub['label']

df_val['mbert'] = df_mbert['label']
# df_val['mbert_sub'] = df_mbert_sub['label']

df_val['roberta'] = df_roberta['label']
# df_val['roberta_sub'] = df_roberta_sub['label']

df_val['mlm'] = df_mlm['label']

print(df_val.head())

                                                text  label  sagor  mbert  \
0  পাডা পুতার মাঝখানে পরে সাধারণ ২ মানুষের জিবন শ...      0      0      1   
1  করোনার চাপে অনেক কিছু বন্ধ ও অনেক বিধি নিষেধ ক...      0      0      0   
2  সঠিক তদন্ত করতে হবে। বিচারের আওতায় আনতে হবে য...      0      0      0   
3  যে লোকটা মারা গেছে তার কি হবে তার দায়ভার কে ন...      0      0      0   
4  নিউ মার্কেট এবং গুলিস্থান মার্কেটের ব্যবসায়ীর...      1      0      2   

   roberta  mlm  
0        1    1  
1        0    0  
2        0    0  
3        0    0  
4        2    2  


In [ ]:
!pip install -q tensorflow

In [ ]:
import tensorflow as tf

In [ ]:
from sklearn.model_selection import train_test_split
X = df_val.drop(['text','label'], axis=1)
y = df_val['label']
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.5, stratify=y, random_state=42
)
X_train,y_train, X_test, y_test

(      sagor  mbert  roberta  mlm
 412       0      0        0    0
 1237      0      0        0    0
 39        0      0        0    0
 1063      0      0        2    2
 182       1      1        1    1
 ...     ...    ...      ...  ...
 882       0      0        0    0
 513       0      0        0    0
 223       0      1        0    0
 426       0      0        0    0
 268       0      0        1    0
 
 [665 rows x 4 columns],
 412     0
 1237    0
 39      0
 1063    2
 182     1
        ..
 882     0
 513     2
 223     0
 426     0
 268     0
 Name: label, Length: 665, dtype: int64,
       sagor  mbert  roberta  mlm
 157       0      0        2    0
 378       1      1        2    1
 322       1      1        1    1
 229       1      1        1    1
 1285      0      0        0    0
 ...     ...    ...      ...  ...
 406       0      0        0    0
 348       0      0        0    0
 1318      1      1        1    1
 1098      0      0        0    0
 686       0      0        0 

In [ ]:
 y_test.value_counts()

0    359
1    208
2     98
Name: label, dtype: int64

In [ ]:
# tf.random.set_seed(42)
# model = tf.keras.Sequential([
#     tf.keras.layers.Dense(128, activation='relu'),
#     tf.keras.layers.Dense(256, activation='relu'),
#     tf.keras.layers.Dense(256, activation='relu'),
#     tf.keras.layers.Dense(3, activation='sigmoid')
# ])
# model.compile(
#     loss=tf.keras.losses.sparse_categorical_crossentropy,
#     optimizer=tf.keras.optimizers.Adam(lr=0.03),
#     metrics=[
#         tf.keras.metrics.BinaryAccuracy(name='accuracy'),
#         tf.keras.metrics.Precision(name='precision'),
#         tf.keras.metrics.Recall(name='recall')
#     ]
# )
# history = model.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test))

In [ ]:
import tensorflow as tf
from sklearn.metrics import f1_score
import numpy as np

# Define a custom macro F1 metric function
def macro_f1_metric(y_true, y_pred):
    y_pred_labels = tf.argmax(y_pred, axis=1)
    return tf.py_function(lambda y_true, y_pred_labels: f1_score(y_true, y_pred_labels, average='macro'),
                          (y_true, y_pred_labels), tf.float32)

tf.random.set_seed(42)
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
])
model.compile(
    loss=tf.keras.losses.sparse_categorical_crossentropy,
    optimizer=tf.keras.optimizers.Adam(lr=0.03),
    metrics=[
        macro_f1_metric,  # Use the custom macro F1 metric
    ]
)

# # Split your data into training and validation sets
# from sklearn.model_selection import train_test_split
# X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Define a custom callback to save the best model based on macro F1 score
class MacroF1Callback(tf.keras.callbacks.Callback):
    def __init__(self, validation_data):
        super().__init__()
        self.validation_data = validation_data
        self.best_macro_f1 = 0.0

    def on_epoch_end(self, epoch, logs=None):
        # print(self.validation_data[0])
        val_predictions = self.model.predict(self.validation_data[0])
        val_true = self.validation_data[1]
        val_macro_f1 = f1_score(val_true, np.argmax(val_predictions, axis=1), average='macro')

        if val_macro_f1 > self.best_macro_f1:
            self.best_macro_f1 = val_macro_f1
            self.model.save('best_model_macro_f1.h5')
            print('saved best model')

# Create the Macro F1 callback
macro_f1_callback = MacroF1Callback(validation_data=(X_test, y_test))

history = model.fit(
    X_train, y_train,
    epochs=30,
    validation_data=(X_test, y_test),
    callbacks=[macro_f1_callback]  # Use the custom Macro F1 callback
)


Epoch 1/30
21/21 [==============================] - 0s 2ms/step
saved best model
21/21 [==============================] - 2s 33ms/step - loss: 0.9714 - macro_f1_metric: 0.5414 - val_loss: 0.8045 - val_macro_f1_metric: 0.5148
Epoch 2/30
21/21 [==============================] - 0s 2ms/step
saved best model
21/21 [==============================] - 0s 20ms/step - loss: 0.7513 - macro_f1_metric: 0.5759 - val_loss: 0.6508 - val_macro_f1_metric: 0.5876
Epoch 3/30
21/21 [==============================] - 0s 22ms/step - loss: 0.6766 - macro_f1_metric: 0.6290 - val_loss: 0.6838 - val_macro_f1_metric: 0.5007
Epoch 4/30
21/21 [==============================] - 0s 2ms/step
saved best model
21/21 [==============================] - 0s 20ms/step - loss: 0.6597 - macro_f1_metric: 0.6356 - val_loss: 0.6050 - val_macro_f1_metric: 0.7492
Epoch 5/30
21/21 [==============================] - 0s 2ms/step
saved best model
21/21 [==============================] - 0s 19ms/step - loss: 0.6310 - macro_f1_metric: 0

In [ ]:
# Load the best model based on macro F1 score at the end of training
best_model_macro_f1 = tf.keras.models.load_model('best_model_macro_f1.h5',
                                                 custom_objects={'macro_f1_metric': macro_f1_metric})

In [ ]:
sample_input = np.array([1, 1, 0, 1])  # Example input with 3 features

# 1. Preprocess the input data (apply the same preprocessing used during training)
# ... (apply your preprocessing steps here if needed)

# 2. Reshape the input data to create a batch of size 1
sample_input = np.expand_dims(sample_input, axis=0)

# 3. Use the model to make predictions
predictions = best_model_macro_f1.predict(sample_input)
print(np.argmax(predictions))

1/1 [==============================] - 0s 67ms/step
1


In [ ]:
df_test = pd.read_csv("test (3).csv")

df_sagor_test = pd.read_csv("/content/sagor_test.csv")
df_sagor_sub_test = pd.read_csv("/content/sagor_test_sub.csv")

df_mbert_test = pd.read_csv("/content/mbert_test.csv")
df_mbert_sub_test = pd.read_csv("/content/mbert_test_sub.csv")

df_roberta_test = pd.read_csv("/content/roberta_test.csv")
df_roberta_sub_test = pd.read_csv("/content/roberta_test_sub.csv")

df_mlm_test = pd.read_csv("/content/mlm_test.csv")

print(df_mbert_sub_test)

                                                   text  label
0     বাংলাদেশের হিন্দুরা নিজেদের জন্য আলাদা হিন্দু ...      0
1     মাইজদী - চৌমুহুনী - ফেনী মন্দিরে হামলা নিয়ে রি...      2
2     দয়া করে পবিত্র কুরআনুল কারিম বলেন,,,,পবিত্র কথ...      0
3     বিবিসি হলো সত্য কে বিনষ্টকারী আর মিথ্যা কে গ্র...      0
4               বুধবার কি তোরা মারা গেছিলি বিবিসি বাংলা      0
...                                                 ...    ...
2011  নিজের ধর্মের পরিধানটি কোনো ধর্ম স্থানেই ভালো ম...      0
2012  হিন্দু মহিলাদের ওপরও বেদের মতে পর্দা করা আবশ্য...      1
2013                           ভারত ধ্বংস হবে অতি দ্রুত      2
2014  ভারতীয় উপমহাদেশে উপর আল্লাহর গজব নাজিল হোক আমি...      2
2015  মুসলিম আল্লাহ চাড়া কাউকে ভয় করে না।। আল্লাহুআক...      0

[2016 rows x 2 columns]


In [ ]:
# Add columns to the test dataframe
df_test['sagor'] = df_sagor_test['label']
df_test['sagor_sub'] = df_sagor_sub_test['label']

df_test['mbert'] = df_mbert_test['label']
df_test['mbert_sub'] = df_mbert_sub_test['label']

df_test['roberta'] = df_roberta_test['label']
df_test['roberta_sub'] = df_roberta_sub_test['label']

df_test['mlm'] = df_mlm_test['label']

# Print the first few rows of the test dataframe
print(df_test.head())

                                                text  label  sagor  sagor_sub  \
0  বাংলাদেশের হিন্দুরা নিজেদের জন্য আলাদা হিন্দু ...      0      1          1   
1  মাইজদী - চৌমুহুনী - ফেনী মন্দিরে হামলা নিয়ে রি...      0      1          0   
2  দয়া করে পবিত্র কুরআনুল কারিম বলেন,,,,পবিত্র কথ...      0      0          0   
3  বিবিসি হলো সত্য কে বিনষ্টকারী আর মিথ্যা কে গ্র...      1      1          0   
4            বুধবার কি তোরা মারা গেছিলি বিবিসি বাংলা      1      1          1   

   mbert  mbert_sub  roberta  roberta_sub  mlm  
0      0          0        2            0    2  
1      1          2        2            2    0  
2      0          0        1            0    0  
3      1          0        0            0    1  
4      1          0        1            0    0  


In [ ]:
stacked_pred=[]
for i in range(len(df_test['label'])):
  sample_input = np.array([
    df_test['sagor'][i],        # Assign value from 'sagor' column
    # df_test['sagor_sub'][i],    # Assign value from 'sagor_sub' column
    df_test['mbert'][i],        # Assign value from 'mbert' column
    # df_test['mbert_sub'][i],    # Assign value from 'mbert_sub' column
    df_test['roberta'][i],      # Assign value from 'roberta' column
    # df_test['roberta_sub'][i],  # Assign value from 'roberta_sub' column
    df_test['mlm'][i]           # Assign value from 'mlm' column
  ])
  # Example input with 3 features

  # 1. Preprocess the input data (apply the same preprocessing used during training)
  # ... (apply your preprocessing steps here if needed)

  # 2. Reshape the input data to create a batch of size 1
  sample_input = np.expand_dims(sample_input, axis=0)

  # 3. Use the model to make predictions
  predictions = best_model_macro_f1.predict(sample_input)
  # print(np.argmax(predictions))
  stacked_pred.append(np.argmax(predictions))

1/1 [==============================] - 0s 31ms/step


In [ ]:
import pandas as pd

# Load each csv file into a separate dataframe
df1 = df_mlm_test
df2 = df_sagor_test
df3 = df_sagor_sub_test
df4 = df_mbert_test
df5 = df_mbert_sub_test
df6 = df_roberta_test
df7 = df_roberta_sub_test

# Extract the label columns from each dataframe
label1 = df1.loc[1, 'label']
label2 = df2.iloc[:, 0]
label3 = df3.iloc[:, 0]
label4 = df4.iloc[:, 0]
label5 = df5.iloc[:, 0]
label6 = df6.iloc[:, 0]
label7 = df7.iloc[:, 0]

preds=[]
print(len(df1))
for i in range(len(df1)):
  l=[0,0,0]
  label1 = df1.loc[i, 'label']
  label2 = df2.loc[i, 'label']
  label3 = df3.loc[i, 'label']
  label4 = df4.loc[i, 'label']
  label5 = df5.loc[i, 'label']
  label6 = df6.loc[i, 'label']
  label7 = df7.loc[i, 'label']
  l[label1]+=3
  l[label2]+=1
  l[label3]+=1
  l[label4]+=1
  l[label5]+=1
  l[label6]+=1
  l[label7]+=1
  preds.append(l.index(max(l)))
print(preds)
print(len(preds))

2016
[2, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 2, 0, 0, 0, 0, 0, 2, 2, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 2, 0, 1, 2, 0, 0, 0, 0, 1, 0, 0, 1, 2, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 2, 1, 0, 0, 0, 1, 2, 0, 0, 0, 1, 0, 0, 0, 1, 0, 2, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 2, 0, 0, 1, 0, 0, 1, 0, 0, 1, 2, 1, 0, 0, 2, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 2, 1, 1, 1, 2, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 2, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 2, 2, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 2, 0, 2, 0, 1, 0, 0, 1, 0, 0, 1, 1, 2, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 2, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 2, 1, 1, 0, 0, 0, 0, 0, 1, 0, 2, 1, 1, 0, 0, 0, 1, 2, 2, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 2, 2, 0, 0, 2, 0, 1, 0, 1, 1

In [ ]:
# stacked_pred = preds

tp = [0, 0, 0]
tn = [0, 0, 0]
fp = [0, 0, 0]
fn = [0, 0, 0]

for i in range(len(df_test['label'])):
  for label in [0, 1, 2]:
      if df_test['label'][i] == label and stacked_pred[i] == label:
          tp[label] += 1
      elif df_test['label'][i] != label and stacked_pred[i] != label:
          tn[label] += 1
      elif df_test['label'][i] != label and stacked_pred[i] == label:
          fp[label] += 1
      elif df_test['label'][i] == label and stacked_pred[i] != label:
          fn[label] += 1

for label in [0, 1, 2]:
    print(f"Label {label}:")
    print(f"True Positives (TP): {tp[label]}")
    print(f"True Negatives (TN): {tn[label]}")
    print(f"False Positives (FP): {fp[label]}")
    print(f"False Negatives (FN): {fn[label]}")
    print()

Label 0:
True Positives (TP): 941
True Negatives (TN): 691
False Positives (FP): 229
False Negatives (FN): 155

Label 1:
True Positives (TP): 394
True Negatives (TN): 1209
False Positives (FP): 88
False Negatives (FN): 325

Label 2:
True Positives (TP): 171
True Negatives (TN): 1622
False Positives (FP): 193
False Negatives (FN): 30



In [ ]:
f1 = []
for label in [0, 1, 2]:
    precision = tp[label] / (tp[label] + fp[label])
    recall = tp[label] / (tp[label] + fn[label])
    f1.append(2 * (precision * recall) / (precision + recall))

# Calculate macro F1
macro_f1 = sum(f1) / len(f1)

print(f"Macro F1: {macro_f1}")

Macro F1: 0.6973226760805761


In [ ]:
stacked_df = pd.read_csv("test (3).csv")
stacked_df = stacked_df.drop(['label'], axis=1)
print(stacked_df)

                                                   text
0     বাংলাদেশের হিন্দুরা নিজেদের জন্য আলাদা হিন্দু ...
1     মাইজদী - চৌমুহুনী - ফেনী মন্দিরে হামলা নিয়ে রি...
2     দয়া করে পবিত্র কুরআনুল কারিম বলেন,,,,পবিত্র কথ...
3     বিবিসি হলো সত্য কে বিনষ্টকারী আর মিথ্যা কে গ্র...
4               বুধবার কি তোরা মারা গেছিলি বিবিসি বাংলা
...                                                 ...
2011  নিজের ধর্মের পরিধানটি কোনো ধর্ম স্থানেই ভালো ম...
2012  হিন্দু মহিলাদের ওপরও বেদের মতে পর্দা করা আবশ্য...
2013                           ভারত ধ্বংস হবে অতি দ্রুত
2014  ভারতীয় উপমহাদেশে উপর আল্লাহর গজব নাজিল হোক আমি...
2015  মুসলিম আল্লাহ চাড়া কাউকে ভয় করে না।। আল্লাহুআক...

[2016 rows x 1 columns]


In [ ]:
stacked_df['label'] = stacked_pred
print(stacked_df)
stacked_df.to_csv('stacking.csv')

                                                   text  label
0     বাংলাদেশের হিন্দুরা নিজেদের জন্য আলাদা হিন্দু ...      2
1     মাইজদী - চৌমুহুনী - ফেনী মন্দিরে হামলা নিয়ে রি...      0
2     দয়া করে পবিত্র কুরআনুল কারিম বলেন,,,,পবিত্র কথ...      0
3     বিবিসি হলো সত্য কে বিনষ্টকারী আর মিথ্যা কে গ্র...      1
4               বুধবার কি তোরা মারা গেছিলি বিবিসি বাংলা      0
...                                                 ...    ...
2011  নিজের ধর্মের পরিধানটি কোনো ধর্ম স্থানেই ভালো ম...      0
2012  হিন্দু মহিলাদের ওপরও বেদের মতে পর্দা করা আবশ্য...      2
2013                           ভারত ধ্বংস হবে অতি দ্রুত      2
2014  ভারতীয় উপমহাদেশে উপর আল্লাহর গজব নাজিল হোক আমি...      2
2015  মুসলিম আল্লাহ চাড়া কাউকে ভয় করে না।। আল্লাহুআক...      0

[2016 rows x 2 columns]
